In [4]:
!pip install dgl-cu111 -f https://data.dgl.ai/wheels/repo.html

Looking in links: https://data.dgl.ai/wheels/repo.html
     |████████████████████████████████| 165.0 MB 46 kB/s 


In [ ]:
import torch
torch.version.cuda

'11.1'

In [22]:
%%writefile train_ppi_custom.py
import argparse
from os import path
from networkx.algorithms.components.attracting import number_attracting_components
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn.functional as F
from dgl import batch
from dgl.data.ppi import LegacyPPIDataset
from dgl.nn.pytorch import GraphConv
# from dgl.nn.pytorch import GATConv 
import dgl.function as fn
from dgl.nn import GATConv
# from torch_geometric.nn import GATConv as GConv
from sklearn.metrics import f1_score
from torch import nn, optim
from torch.utils.data import DataLoader
import pandas as pd


MODEL_STATE_FILE = "/content/model_state_1.pth"


class GAT(nn.Module):
    def __init__(self,
                 g,
                 num_layers,
                 in_dim,
                 num_hidden,
                 num_classes,
                 heads,
                 activation,
                 feat_drop,
                 attn_drop,
                 negative_slope,
                 residual):
        super(GAT, self).__init__()
        self.g = g
        self.num_layers = num_layers
        self.layers = nn.ModuleList()
        self.activation = activation
        # input projection (no residual)
        self.layers.append(GATConv(
            in_dim, num_hidden, heads[0],
            feat_drop, attn_drop, negative_slope, False, self.activation))
        # hidden layers
        for l in range(1, num_layers):
            # due to multi-head, the in_dim = num_hidden * num_heads
            self.layers.append(GATConv(
                num_hidden * heads[l-1], num_hidden, heads[l],
                feat_drop, attn_drop, negative_slope, residual, self.activation))
        # output projection
        self.layers.append(GATConv(
            num_hidden * heads[-2], num_classes, heads[-1],
            feat_drop, attn_drop, negative_slope, residual, None))

    def forward(self, inputs):
        h = inputs
        for l in range(self.num_layers):
            h = self.layers[l](self.g, h).flatten(1)
        # output projection
        logits = self.layers[-1](self.g, h).mean(1)
        return logits

def main(args):

    # load dataset and create dataloader
    train_dataset, test_dataset = LegacyPPIDataset(mode="train"), LegacyPPIDataset(mode="test")
    train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, collate_fn=collate_fn)
    test_dataloader = DataLoader(test_dataset, batch_size=args.batch_size, collate_fn=collate_fn)
    n_features, n_classes = train_dataset.features.shape[1], train_dataset.labels.shape[1]
    
    # create the model, loss function and optimizer
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    ########### Replace this model with your own GNN implemented class ################################
    hiddens = [256, 512, 1024]
    heds = [3, 8] 
    activations = [F.relu, F.elu, F.leaky_relu, torch.tanh]
    fdrop = [0.1, 0.6]
    adrop = [0.1, 0.6]
    nslope = [0.01, 0.2, 0.5]
    losses = [nn.BCEWithLogitsLoss(), nn.MultiLabelSoftMarginLoss()]
    optimizers = [torch.optim.Adagrad, torch.optim.Adam, torch.optim.RMSprop]
    
    final = []
    i = 0 #Grid Search 
    for size in hiddens:
      for h in heds:
        for a in activations:
          for f in fdrop:
            for attn in adrop:
              for n in nslope:
                for loss in losses:
                  for optimizer in optimizers:
                    model = GAT(g=train_dataset.graph, num_layers=2, in_dim=n_features,
                                            num_hidden=size, num_classes=n_classes, heads=[h,h,h],
                                            activation = a, feat_drop =f, attn_drop =attn,
                                            negative_slope=n, residual=False).to(device)
                    ###################################################################################################
                    
                    loss_fcn = loss
                    optimizer_ = optimizer(model.parameters())
                    try:
                      # train
                      if args.mode == "train":
                          train(model, loss_fcn, device, optimizer_, train_dataloader, test_dataset)
                          torch.save(model.state_dict(), MODEL_STATE_FILE)

                      # import model from file
                      model.load_state_dict(torch.load(MODEL_STATE_FILE))

                      # test the model
                      score = test(model, loss_fcn, device, test_dataloader)
                      
                      o = str(optimizer).split(".")[-1][:-2]
                      l = str(loss)[:-2]
                      act = "tanh" if str(a).split(" ")[1] == "method" else  str(a).split(" ")[1]
                      temp = [size, h, act, f, attn, n, l, o, score]
                      final.append(temp)
                      print(temp)

                      i += 1 #Save every 100 models
                      if i % 100 == 0: 
                          df = pd.DataFrame(final, columns = ["num_hidden", "heads", "activation", 
                                              "feat_drop", "attn_drop", "negative_slope", 
                                              "loss_fcn", "optimizer", "score"])
                          df.to_csv(f"df_{i}.csv")
                    except: 
                      i+=1
                      print([size, h, str(a), f, attn, n, str(loss), str(optimizer).split(".")[-1][:-2], score], "failed")
                      pass
   
   #make and save the final list of all combinations
    df = pd.DataFrame(final, columns = ["num_hidden", "heads", "activation", 
                                        "feat_drop", "attn_drop", "negative_slope", 
                                        "loss_fcn", "optimizer", "score"])
    df.to_csv("df.csv")
    return model

def train(model, loss_fcn, device, optimizer, train_dataloader, test_dataset):

    f1_score_list = []
    epoch_list = []

    for epoch in range(args.epochs):
        model.train()
        losses = []
        for batch, data in enumerate(train_dataloader):
            subgraph, features, labels = data
            subgraph = subgraph.to(device)
            features = features.to(device)
            labels = labels.to(device)
            model.g = subgraph
            for layer in model.layers:
                layer.g = subgraph
            logits = model(features.float())
            loss = loss_fcn(logits, labels.float())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            losses.append(loss.item())
        loss_data = np.array(losses).mean()
        # print("Epoch {:05d} | Loss: {:.4f}".format(epoch + 1, loss_data))

        if epoch % 5 == 0:
            scores = []
            for batch, test_data in enumerate(test_dataset):
                subgraph, features, labels = test_data
                subgraph = subgraph.clone().to(device)
                features = features.clone().detach().to(device)
                labels = labels.clone().detach().to(device)
                score, _ = evaluate(features.float(), model, subgraph, labels.float(), loss_fcn)
                scores.append(score)
                f1_score_list.append(score)
                epoch_list.append(epoch)
            # print("F1-Score: {:.4f} ".format(np.array(scores).mean()))
        
        if (epoch == 50 and np.array(scores).mean() < 0.5) or \
            (epoch == 150 and np.array(scores).mean() < 0.6) or \
            (epoch == 200 and np.array(scores).mean() < 0.7):
            print(epoch, np.array(scores).mean())
            return

    # plot_f1_score(epoch_list, f1_score_list)

def test(model, loss_fcn, device, test_dataloader):
    test_scores = []
    for batch, test_data in enumerate(test_dataloader):
        subgraph, features, labels = test_data
        subgraph = subgraph.to(device)
        features = features.to(device)
        labels = labels.to(device)
        test_scores.append(evaluate(features, model, subgraph, labels.float(), loss_fcn)[0])
    mean_scores = np.array(test_scores).mean()
    # print("F1-Score: {:.4f}".format(np.array(test_scores).mean()))
    return mean_scores

def evaluate(features, model, subgraph, labels, loss_fcn):
    with torch.no_grad():
        model.eval()
        model.g = subgraph
        for layer in model.layers:
            layer.g = subgraph
        output = model(features.float())
        loss_data = loss_fcn(output, labels.float())
        predict = np.where(output.data.cpu().numpy() >= 0.5, 1, 0)
        score = f1_score(labels.data.cpu().numpy(), predict, average="micro")
        return score, loss_data.item()

def collate_fn(sample) :
    # concatenate graph, features and labels w.r.t batch size
    graphs, features, labels = map(list, zip(*sample))
    graph = batch(graphs)
    features = torch.from_numpy(np.concatenate(features))
    labels = torch.from_numpy(np.concatenate(labels))
    return graph, features, labels

def plot_f1_score(epoch_list, f1_score_list) :

    plt.plot(epoch_list, f1_score_list)
    plt.title("Evolution of f1 score w.r.t epochs")
    plt.show()

if __name__ == "__main__":

    # PARSER TO ADD OPTIONS
    parser = argparse.ArgumentParser()
    parser.add_argument("--mode",  choices=["train", "test"], default="train")
    # parser.add_argument("--gpu", type=int, default=-1, help="GPU to use. Set -1 to use CPU.")
    parser.add_argument("--epochs", type=int, default=250)
    parser.add_argument("--batch-size", type=int, default=2)
    args = parser.parse_args()

    # READ MAIN
    main(args)


Overwriting train_ppi_custom.py


In [ ]:
!python3 train_ppi_custom.py --mode train

Using backend: pytorch
/usr/local/lib/python3.7/dist-packages/dgl/data/utils.py:286: UserWarning: Property dataset.features will be deprecated, please use dataset.graphs[i].ndata['feat'] instead.
  warnings.warn('Property {} will be deprecated, please use {} instead.'.format(old, new))
/usr/local/lib/python3.7/dist-packages/dgl/data/utils.py:286: UserWarning: Property dataset.labels will be deprecated, please use dataset.graphs[i].ndata['label'] instead.
  warnings.warn('Property {} will be deprecated, please use {} instead.'.format(old, new))
50 0.3816100274985258
[256, 3, 'relu', 0.1, 0.1, 0.01, 'BCEWithLogitsLoss', 'Adagrad', 0.38108908293670124]
50 0.4863467253807959
[256, 3, 'relu', 0.1, 0.1, 0.01, 'BCEWithLogitsLoss', 'Adam', 0.48371222647057793]
50 0.33052555887985197
[256, 3, 'relu', 0.1, 0.1, 0.01, 'BCEWithLogitsLoss', 'RMSprop', 0.33059347192584815]
50 0.3246956306275264
[256, 3, 'relu', 0.1, 0.1, 0.01, 'MultiLabelSoftMarginLoss', 'Adagrad', 0.32282446304358303]
100 0.5867701

In [ ]:
!python3 train_ppi_custom.py --mode test

Using backend: pytorch
/usr/local/lib/python3.7/dist-packages/dgl/data/utils.py:286: UserWarning: Property dataset.features will be deprecated, please use dataset.graphs[i].ndata['feat'] instead.
  warnings.warn('Property {} will be deprecated, please use {} instead.'.format(old, new))
/usr/local/lib/python3.7/dist-packages/dgl/data/utils.py:286: UserWarning: Property dataset.labels will be deprecated, please use dataset.graphs[i].ndata['label'] instead.
  warnings.warn('Property {} will be deprecated, please use {} instead.'.format(old, new))
Traceback (most recent call last):
  File "train_ppi_custom.py", line 115, in main
    model.load_state_dict(torch.load(MODEL_STATE_FILE))
  File "/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py", line 1483, in load_state_dict
    self.__class__.__name__, "\n\t".join(error_msgs)))
RuntimeError: Error(s) in loading state_dict for GAT:
	size mismatch for layers.0.attn_l: copying a param with shape torch.Size([1, 2, 4]) from checkp

# OG Model

In [ ]:
%%writefile train_ppi_baseline.py
import argparse
from os import path

import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn.functional as F
from dgl import batch
from dgl.data.ppi import LegacyPPIDataset
from dgl.nn.pytorch import GraphConv
from dgl.nn.pytorch import GATConv 
import dgl.function as fn
from dgl.nn import GATConv
# from torch_geometric.nn import GATConv as GConv
from sklearn.metrics import f1_score
from torch import nn, optim
from torch.utils.data import DataLoader
import pandas as pd


MODEL_STATE_FILE = "/content/model_state.pth"

class BasicGraphModel(nn.Module):

    def __init__(self, g, n_layers, input_size, hidden_size, output_size, nonlinearity):
        super().__init__()

        self.g = g
        self.layers = nn.ModuleList()
        self.layers.append(GraphConv(input_size, hidden_size, activation=nonlinearity))
        for i in range(n_layers - 1):
            self.layers.append(GraphConv(hidden_size, hidden_size, activation=nonlinearity))
        self.layers.append(GraphConv(hidden_size, output_size))

    def forward(self, inputs):
        outputs = inputs
        for i, layer in enumerate(self.layers):
            outputs = layer(self.g, outputs)
        return outputs

def main(args):

    # load dataset and create dataloader
    train_dataset, test_dataset = LegacyPPIDataset(mode="train"), LegacyPPIDataset(mode="test")
    train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, collate_fn=collate_fn)
    test_dataloader = DataLoader(test_dataset, batch_size=args.batch_size, collate_fn=collate_fn)
    n_features, n_classes = train_dataset.features.shape[1], train_dataset.labels.shape[1]
    # create the model, loss function and optimizer
    # print(train_dataset.labels, train_dataset.features)
    # print(train_dataset.labels, train_dataset.features)
    # device = torch.device("cpu" if args.gpu < 0 else "cuda:" + str(args.gpu))
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    ########### Replace this model with your own GNN implemented class ################################

    model = BasicGraphModel(g=train_dataset.graph, n_layers=2, input_size=n_features,
                            hidden_size=256, output_size=n_classes, nonlinearity=F.elu).to(device)

    ###################################################################################################

    loss_fcn = nn.BCEWithLogitsLoss()
    o = torch.optim.Adam
    optimizer = o(model.parameters())

    # train
    if args.mode == "train":
        train(model, loss_fcn, device, optimizer, train_dataloader, test_dataset)
        torch.save(model.state_dict(), MODEL_STATE_FILE)

    # import model from file
    model.load_state_dict(torch.load(MODEL_STATE_FILE))

    # test the model
    score = test(model, loss_fcn, device, test_dataloader)
    final = []
    temp = [256, 1, str(F.relu), 0, 0, 0.2, str(nn.BCEWithLogitsLoss()), str(o), score]
    final.append(temp)
    df = pd.DataFrame(final, columns = ["num_hidden", "heads", "activation", 
                                        "feat_drop", "attn_drop", "negative_slope", 
                                        "loss_fcn", "optimizer", "score"])
    df.to_csv("df.csv")
    return model

def train(model, loss_fcn, device, optimizer, train_dataloader, test_dataset):

    f1_score_list = []
    epoch_list = []

    for epoch in range(args.epochs):
        model.train()
        losses = []
        for batch, data in enumerate(train_dataloader):
            subgraph, features, labels = data
            subgraph = subgraph.to(device)
            features = features.to(device)
            labels = labels.to(device)
            model.g = subgraph
            for layer in model.layers:
                layer.g = subgraph
            logits = model(features.float())
            loss = loss_fcn(logits, labels.float())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            losses.append(loss.item())
        loss_data = np.array(losses).mean()
        print("Epoch {:05d} | Loss: {:.4f}".format(epoch + 1, loss_data))

        if epoch % 5 == 0:
            scores = []
            for batch, test_data in enumerate(test_dataset):
                subgraph, features, labels = test_data
                subgraph = subgraph.clone().to(device)
                features = features.clone().detach().to(device)
                labels = labels.clone().detach().to(device)
                score, _ = evaluate(features.float(), model, subgraph, labels.float(), loss_fcn)
                scores.append(score)
                f1_score_list.append(score)
                epoch_list.append(epoch)
            print("F1-Score: {:.4f} ".format(np.array(scores).mean()))

    plot_f1_score(epoch_list, f1_score_list)

def test(model, loss_fcn, device, test_dataloader):
    test_scores = []
    for batch, test_data in enumerate(test_dataloader):
        subgraph, features, labels = test_data
        subgraph = subgraph.to(device)
        features = features.to(device)
        labels = labels.to(device)
        test_scores.append(evaluate(features, model, subgraph, labels.float(), loss_fcn)[0])
    mean_scores = np.array(test_scores).mean()
    print("F1-Score: {:.4f}".format(np.array(test_scores).mean()))
    return mean_scores

def evaluate(features, model, subgraph, labels, loss_fcn):
    with torch.no_grad():
        model.eval()
        model.g = subgraph
        for layer in model.layers:
            layer.g = subgraph
        output = model(features.float())
        loss_data = loss_fcn(output, labels.float())
        predict = np.where(output.data.cpu().numpy() >= 0.5, 1, 0)
        score = f1_score(labels.data.cpu().numpy(), predict, average="micro")
        return score, loss_data.item()

def collate_fn(sample) :
    # concatenate graph, features and labels w.r.t batch size
    graphs, features, labels = map(list, zip(*sample))
    graph = batch(graphs)
    features = torch.from_numpy(np.concatenate(features))
    labels = torch.from_numpy(np.concatenate(labels))
    return graph, features, labels

def plot_f1_score(epoch_list, f1_score_list) :

    plt.plot(epoch_list, f1_score_list)
    plt.title("Evolution of f1 score w.r.t epochs")
    plt.show()

if __name__ == "__main__":

    # PARSER TO ADD OPTIONS
    parser = argparse.ArgumentParser()
    parser.add_argument("--mode",  choices=["train", "test"], default="train")
    # parser.add_argument("--gpu", type=int, default=-1, help="GPU to use. Set -1 to use CPU.")
    parser.add_argument("--epochs", type=int, default=250)
    parser.add_argument("--batch-size", type=int, default=2)
    args = parser.parse_args()

    # READ MAIN
    main(args)


Overwriting train_ppi_baseline.py


In [ ]:
!python3 train_ppi_baseline.py --mode train

Using backend: pytorch
/usr/local/lib/python3.7/dist-packages/dgl/data/utils.py:286: UserWarning: Property dataset.features will be deprecated, please use dataset.graphs[i].ndata['feat'] instead.
  warnings.warn('Property {} will be deprecated, please use {} instead.'.format(old, new))
/usr/local/lib/python3.7/dist-packages/dgl/data/utils.py:286: UserWarning: Property dataset.labels will be deprecated, please use dataset.graphs[i].ndata['label'] instead.
  warnings.warn('Property {} will be deprecated, please use {} instead.'.format(old, new))
Epoch 00001 | Loss: 0.6865
F1-Score: 0.0017 
Epoch 00002 | Loss: 0.6504
Epoch 00003 | Loss: 0.6064
Epoch 00004 | Loss: 0.5917
Epoch 00005 | Loss: 0.5807
Epoch 00006 | Loss: 0.5733
F1-Score: 0.3356 
Epoch 00007 | Loss: 0.5675
Epoch 00008 | Loss: 0.5635
Epoch 00009 | Loss: 0.5606
Epoch 00010 | Loss: 0.5584
Epoch 00011 | Loss: 0.5567
F1-Score: 0.3639 
Epoch 00012 | Loss: 0.5552
Epoch 00013 | Loss: 0.5538
Epoch 00014 | Loss: 0.5525
Epoch 00015 | Loss

In [ ]:
!python3 train_ppi_baseline.py --mode test

Using backend: pytorch
/usr/local/lib/python3.7/dist-packages/dgl/data/utils.py:286: UserWarning: Property dataset.features will be deprecated, please use dataset.graphs[i].ndata['feat'] instead.
  warnings.warn('Property {} will be deprecated, please use {} instead.'.format(old, new))
/usr/local/lib/python3.7/dist-packages/dgl/data/utils.py:286: UserWarning: Property dataset.labels will be deprecated, please use dataset.graphs[i].ndata['label'] instead.
  warnings.warn('Property {} will be deprecated, please use {} instead.'.format(old, new))
F1-Score: 0.4711


In [ ]:
data = [""]
z = pd.DataFrame()